In [1]:
import boto3
from boto3.dynamodb.conditions import Key, Attr

dynamoDB = boto3.resource("dynamodb")
table = dynamoDB.Table("demo_table")

### Create Item

In [2]:
table.put_item(
    Item={
        'user_id' : 'USER#002',
        'user_type': 'store_keeper',
        'firstname' : 'Cleetus',
        'lastname' : 'McFarland'
    }
)

{'ResponseMetadata': {'RequestId': 'K192F4CQ7KOFOESQ7CTJHJ06ARVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 22 Jun 2025 16:17:26 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'K192F4CQ7KOFOESQ7CTJHJ06ARVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

### Batch Writer           
* Automatically manage retries, batching into 25 item batches etc..

In [23]:
items = [
    {'user_id': 'USER#003', 'user_type': 'store_keeper', 'firstname': 'Alice', 'age': 30},
    {'user_id': 'USER#004', 'user_type': 'store_keeper', 'firstname': 'Bob', 'age': 25},
    {'user_id': 'USER#005', 'user_type': 'store_keeper', 'firstname': 'Charlie', 'age': 35},
]

with table.batch_writer() as batch:
    for item in items:
        batch.put_item(Item=item)

### Read Item                
* If the table have both a partition key and a sort key both keys must be provided
* If the requested item is not there in the table, response item will be none. 

In [9]:
response = table.get_item(
    Key = {
        'user_id':'USER#001',
        'user_type':'admin'
    }
)
item = response.get('Item')
print(f"item:{item}\nresponse:{response}")

response = table.get_item(
    Key = {
        'user_id':'USER#002',
        'user_type':'admin'
    }
)
item = response.get('Item')
print(f"\n\nitem:{item}\ntype:{type(item)}\nresponse:{response}")

item:{'firstname': 'James', 'lastname': 'Tyler', 'user_id': 'USER#001', 'user_type': 'admin'}
response:{'Item': {'firstname': 'James', 'lastname': 'Tyler', 'user_id': 'USER#001', 'user_type': 'admin'}, 'ResponseMetadata': {'RequestId': 'BKG6PTS0IUIRO1OULQU1IVO3V7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 22 Jun 2025 16:33:47 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '114', 'connection': 'keep-alive', 'x-amzn-requestid': 'BKG6PTS0IUIRO1OULQU1IVO3V7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1272210920'}, 'RetryAttempts': 0}}


item:None
type:<class 'NoneType'>
response:{'ResponseMetadata': {'RequestId': 'PSNH80EP1F3F6AG5I6NCHA1J37VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 22 Jun 2025 16:33:47 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'PSNH80EP1F3F6AG5I6NCHA1J37VV

### Query

#### Partition Key

In [17]:
response = table.query(
    KeyConditionExpression=Key('user_id').eq('USER#001') 
)

for item in response.get('Items'):
    print(item)

{'firstname': 'James', 'lastname': 'Tyler', 'user_id': 'USER#001', 'user_type': 'admin'}


#### Partition Key and Sort Key

In [18]:
response = table.query(
    KeyConditionExpression=Key('user_id').eq('USER#001') & Key('user_type').eq('admin')
)

for item in response.get('Items'):
    print(item)

{'firstname': 'James', 'lastname': 'Tyler', 'user_id': 'USER#001', 'user_type': 'admin'}


#### Partion Key and Other Attributes

In [21]:
response = table.query(
    KeyConditionExpression=Key('user_id').eq('USER#001'),
    FilterExpression = Attr('firstname').eq('James')
)
for item in response.get('Items'):
    print(item)

{'firstname': 'James', 'lastname': 'Tyler', 'user_id': 'USER#001', 'user_type': 'admin'}


### Scan

In [22]:
response = table.scan(
    FilterExpression = Attr('firstname').eq('James')
)
for item in response.get('Items'):
    print(item)

{'firstname': 'James', 'lastname': 'Tyler', 'user_id': 'USER#001', 'user_type': 'admin'}


### Delete Item

In [25]:
table.delete_item(
    Key={
        'user_id':'USER#004',
        'user_type':'store keeper'
    }
)

{'ResponseMetadata': {'RequestId': '1PF5PSLF01GEJKFRMLQD0KQLL7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 22 Jun 2025 17:57:14 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1PF5PSLF01GEJKFRMLQD0KQLL7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

### GIS

### LIS